In [74]:
#!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas chromadb langchain-groq fastembed pypdf openai konlpy sentence-transformers python-docx PyPDF2

In [89]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, SpacyTextSplitter, NLTKTextSplitter, split_text_on_tokens, SentenceTransformersTokenTextSplitter, LatexTextSplitter
from langchain.text_splitter import PythonCodeTextSplitter, KonlpyTextSplitter, ElementType
from langchain.text_splitter import MarkdownTextSplitter

from langchain.document_loaders import PyPDFLoader
import torch
import re
from docx import Document as worddoc

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents.base import Document

import PyPDF2
import pandas as pd
from datasets import Dataset

import PyPDF2
from tqdm.notebook import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document as LangchainDocument

from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

from transformers import AutoTokenizer

In [104]:


class TextChunker:
      def __init__(self):
          pass

      #Extract metadata
      def get_pdf_metadata(self, pdf_path):
          reader = PyPDF2.PdfReader(pdf_path)
          metadata = reader.metadata
          return metadata
      #extract content
      def extract_pdf_content_with_metadata(self, pdf_path: str):
          # Extract the overall PDF metadata
          overall_metadata = self.get_pdf_metadata(pdf_path)

          # Load the PDF and extract text and page-specific metadata using PyPDFLoader
          loader = PyPDFLoader(pdf_path)
          documents = loader.load()
          pages_content = []

          for i, doc in enumerate(tqdm(documents, desc="Processing PDF Pages")):
              page_content = doc.page_content
              page_metadata = doc.metadata

              # Combine overall metadata with page-specific metadata
              combined_metadata = {
                  "source": pdf_path,
                  "page_number": i + 1,
                  **overall_metadata,  # Include overall PDF metadata
                  **page_metadata  # Include existing page-specific metadata
              }

              pages_content.append((page_content, combined_metadata))

          return pages_content

      #lagchain_document
      def create_langchain_document_objects(self,  pdf_content):
          # Create LangchainDocument objects
          RAW_KNOWLEDGE_BASE = [
              LangchainDocument(page_content=text, metadata=metadata) for text, metadata in pdf_content
          ]
          return RAW_KNOWLEDGE_BASE

      #Splitters
      def TextSplitters(self, RAW_KNOWLEDGE_BASE,textsplitter, tokenizer = None, chunk_size = None, chunk_overlap = None, MARKDOWN_SEPARATORS = None , add_start_index = None,strip_whitespace= None ):

          tokenizer_val = tokenizer



          splitter_args = {}

          if chunk_size is not None:
             splitter_args["chunk_size"] = chunk_size
          if chunk_overlap is not None:
             splitter_args["chunk_overlap"] = chunk_overlap
          if strip_whitespace is not None:
             splitter_args["strip_whitespace"] = strip_whitespace #True or False
          if add_start_index is not None:
             splitter_args["add_start_index"] = add_start_index  #True or False

          # splitter_args = {
          #     #"tokenizer": tokenizer,
          #     "chunk_size": chunk_size,
          #     "chunk_overlap": chunk_overlap,
          #     "add_start_index": add_start_index,
          #     "strip_whitespace": strip_whitespace
          # }

          if tokenizer is not None:
             tokenizer = AutoTokenizer.from_pretrained(tokenizer)
             splitter_args["tokenizer"] = tokenizer

          if MARKDOWN_SEPARATORS is not None:
             splitter_args["separators"] = MARKDOWN_SEPARATORS

          print(splitter_args)
          # text_splitter = textsplitter.from_huggingface_tokenizer(
          #         tokenizer,
          #         chunk_size=chunk_size,
          #         chunk_overlap=over_lap_size ,#int(chunk_size / 10),
          #         add_start_index=True,
          #         strip_whitespace=True,
          #         separators=MARKDOWN_SEPARATORS,
          #     )


          if tokenizer_val is not None:
              text_splitter = textsplitter.from_huggingface_tokenizer(**splitter_args)
          else:
              text_splitter = textsplitter(**splitter_args)
          #
          docs_processed = []
          for doc in RAW_KNOWLEDGE_BASE:
            docs_processed += text_splitter.split_documents([doc])

          #print(len(docs_processed))
          return docs_processed

      #Embediding and vectorestore
      def create_embedding_and_vectorstore(self, EMBEDDING_MODEL_NAME, multi_process, normalize_embeddings, docs_processed, persist_directory,collection_name ):
          embedding_model = HuggingFaceEmbeddings(
                            model_name=EMBEDDING_MODEL_NAME,
                            multi_process= multi_process, #True,
                            #model_kwargs={"device": "cuda"},
                            encode_kwargs={"normalize_embeddings": normalize_embeddings },  # set True for cosine similarity
                            )
          KNOWLEDGE_VECTOR_DATABASE = Chroma.from_documents(docs_processed,
                                                            embedding_model,
                                                            persist_directory=  persist_directory, #"/content/drive/MyDrive/chunking/CRAG",
                                                            collection_name=collection_name #"crag"
                                                            )
          #
          KNOWLEDGE_VECTOR_DATABASE.persist()
          #print(len(KNOWLEDGE_VECTOR_DATABASE.get()['documents']))
          return KNOWLEDGE_VECTOR_DATABASE

      #Retrieve content_dic
      def retrieve_content_dic(self, persist_directory, collection_name):
          # Initialize the Chroma database for the specific collection
          knowledge_vector_database = Chroma(
              persist_directory=persist_directory,
              collection_name=collection_name,
          )

          # Retrieve documents from the collection
          knowledge_data = KNOWLEDGE_VECTOR_DATABASE.get()
          documents = knowledge_vector_database.get()['documents']

          organized_data = []

          for i in range(len(knowledge_data['ids'])):
              data_dict = {
                  'id': knowledge_data['ids'][i],
                  'metadata': knowledge_data['metadatas'][i],
                  'document': knowledge_data['documents'][i]
              }
              organized_data.append(data_dict)
          return organized_data

      #Retrieve content_documents
      def retrieve_content_documents(self, persist_directory, collection_name):
          # Initialize the Chroma database for the specific collection
          knowledge_vector_database = Chroma(
              persist_directory=persist_directory,
              collection_name=collection_name,
          )
          # Retrieve documents from the collection
          documents = knowledge_vector_database.get()['documents']
          return documents






#################################################################################
#Usage
text_chunker = TextChunker()

In [101]:
#RecursiveCharacterTextSplitter without tokenizer

pdf_path =r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
pdf_content =   text_chunker.extract_pdf_content_with_metadata(pdf_path)
RAW_KNOWLEDGE_BASE =  text_chunker.create_langchain_document_objects(pdf_content)

# Print the number of pages and the content of the second page
print(len(RAW_KNOWLEDGE_BASE))

print(RAW_KNOWLEDGE_BASE[1].page_content)
print(RAW_KNOWLEDGE_BASE[1].metadata)

MARKDOWN_SEPARATORS = [  #Put MARKDOWN_SEPARATORS = None for non custom delimeter
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

EMBEDDING_MODEL_NAME = "thenlper/gte-small"
chunk_size = 512
over_lap_size = 50 #put zero for non overlap
multi_process = True
normalize_embeddings = True
persist_directory = r"/content/drive/MyDrive/chunking/chunking_chroma_store"
collection_name = "rct_without_emb"

textsplitter = RecursiveCharacterTextSplitter
docs_processed = text_chunker.TextSplitters(RAW_KNOWLEDGE_BASE,textsplitter,   chunk_size = chunk_size ,chunk_overlap =over_lap_size , MARKDOWN_SEPARATORS = MARKDOWN_SEPARATORS )
print(len(docs_processed))

KNOWLEDGE_VECTOR_DATABASE = text_chunker.create_embedding_and_vectorstore(EMBEDDING_MODEL_NAME, multi_process, normalize_embeddings, docs_processed, persist_directory,collection_name )

#retrieved_docs =text_chunker.retrieve_content_documents(persist_directory, collection_name)
retrieved_content = text_chunker.retrieve_content_dic(persist_directory, collection_name)
retrieved_content

Processing PDF Pages:   0%|          | 0/32 [00:00<?, ?it/s]

32
2
{'source': '/content/drive/MyDrive/chunking/ds (1).pdf', 'page_number': 2, '/Author': '', '/Title': '', '/Subject': '', '/Creator': 'LaTeX with hyperref package', '/Producer': 'pdfTeX-1.40.19', '/Keywords': '', '/CreationDate': "D:20190814093040+02'00'", '/ModDate': "D:20190814093040+02'00'", '/Trapped': '/False', '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0', 'page': 1}
{'chunk_size': 512, 'chunk_overlap': 50, 'strip_whitespace': True, 'add_start_index': True, 'separators': ['\n#{1,6} ', '```\n', '\n\\*\\*\\*+\n', '\n---+\n', '\n___+\n', '\n\n', '\n', ' ', '']}
168


[{'id': '0000a089-63ad-4055-849f-9efe4f606522',
  'metadata': {'/Author': '',
   '/CreationDate': "D:20190814093040+02'00'",
   '/Creator': 'LaTeX with hyperref package',
   '/Keywords': '',
   '/ModDate': "D:20190814093040+02'00'",
   '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
   '/Producer': 'pdfTeX-1.40.19',
   '/Subject': '',
   '/Title': '',
   '/Trapped': '/False',
   'page': 29,
   'page_number': 30,
   'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
   'start_index': 449},
  'document': 'P∈U(a,b)⟨P,C(ℓ)⟩−εH(P) where (1.86)\nC(ℓ)def.=∇ED,D′(P(ℓ)) =−D′TP(ℓ)D,\nwhich can be interpreted as a mirror-descent scheme [ ?]. Each update can thus be solved using Sinkhorn\niterations (1.51) with cost C(ℓ). Figure (1.17) illustrates the use of this entropic Gromov-Wasserstein to\ncompute soft maps between domains.\n30'},
 {'id': '00d77c7b-7f0b-49c7-88cf-131d064f0c6a',
  'metadata': {'/Author': '',
   '/CreationDate': 

In [86]:
#RecursiveCharacterTextSplitter wit tokenizer

pdf_path =r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
pdf_content =   text_chunker.extract_pdf_content_with_metadata(pdf_path)
RAW_KNOWLEDGE_BASE =  text_chunker.create_langchain_document_objects(pdf_content)

# Print the number of pages and the content of the second page
print(len(RAW_KNOWLEDGE_BASE))

print(RAW_KNOWLEDGE_BASE[1].page_content)
print(RAW_KNOWLEDGE_BASE[1].metadata)

MARKDOWN_SEPARATORS = [  #Put MARKDOWN_SEPARATORS = [] or  [""] for non custom delimeter
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

EMBEDDING_MODEL_NAME = "thenlper/gte-small"
chunk_size = 512
over_lap_size = 50 #put zero for non overlap
multi_process = True
normalize_embeddings = True
persist_directory = r"/content/drive/MyDrive/chunking/chunking_chroma_store"
collection_name = "rct"

textsplitter = RecursiveCharacterTextSplitter
docs_processed = text_chunker.TextSplitters(RAW_KNOWLEDGE_BASE,textsplitter, tokenizer = EMBEDDING_MODEL_NAME,  chunk_size = chunk_size ,over_lap_size =over_lap_size , MARKDOWN_SEPARATORS = MARKDOWN_SEPARATORS )
print(len(docs_processed))

KNOWLEDGE_VECTOR_DATABASE = text_chunker.create_embedding_and_vectorstore(EMBEDDING_MODEL_NAME, multi_process, normalize_embeddings, docs_processed, persist_directory,collection_name )

#retrieved_docs =text_chunker.retrieve_content_documents(persist_directory, collection_name)
retrieved_content = text_chunker.retrieve_content_dic(persist_directory, collection_name)
retrieved_content

Processing PDF Pages:   0%|          | 0/32 [00:00<?, ?it/s]

32
2
{'source': '/content/drive/MyDrive/chunking/ds (1).pdf', 'page_number': 2, '/Author': '', '/Title': '', '/Subject': '', '/Creator': 'LaTeX with hyperref package', '/Producer': 'pdfTeX-1.40.19', '/Keywords': '', '/CreationDate': "D:20190814093040+02'00'", '/ModDate': "D:20190814093040+02'00'", '/Trapped': '/False', '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0', 'page': 1}
{'chunk_size': 512, 'chunk_overlap': 50, 'add_start_index': True, 'strip_whitespace': True, 'tokenizer': BertTokenizerFast(name_or_path='thenlper/gte-small', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=F

[{'id': '0026ea7b-ce2c-498f-8509-dd85af9693de',
  'metadata': {'/Author': '',
   '/CreationDate': "D:20190814093040+02'00'",
   '/Creator': 'LaTeX with hyperref package',
   '/Keywords': '',
   '/ModDate': "D:20190814093040+02'00'",
   '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
   '/Producer': 'pdfTeX-1.40.19',
   '/Subject': '',
   '/Title': '',
   '/Trapped': '/False',
   'page': 16,
   'page_number': 17,
   'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
   'start_index': 0},
  'document': '-4 -2 0 2 4 6-3-2-101234\nρβραFigure 1.10: Two Gaussians ραandρβ, represented using the contour plots of their densities, with respective\nmean and variance matrices mα= (−2,0),Σα=1\n2(\n1−1\n2;−1\n21)\nandmβ= (3,1),Σβ=(\n2,1\n2;1\n2,1)\n. The\narrows originate at random points xtaken on the plane and end at the corresponding mappings of those\npointsT(x) =mβ+A(x−mα).\n\x00m\nFigure 1.11: Computation of displacement interpo

In [87]:
#CharacterTextSplitter without tokenizer

pdf_path =r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
pdf_content =   text_chunker.extract_pdf_content_with_metadata(pdf_path)
RAW_KNOWLEDGE_BASE =  text_chunker.create_langchain_document_objects(pdf_content)

# Print the number of pages and the content of the second page
print(len(RAW_KNOWLEDGE_BASE))

print(RAW_KNOWLEDGE_BASE[1].page_content)
print(RAW_KNOWLEDGE_BASE[1].metadata)



EMBEDDING_MODEL_NAME = "thenlper/gte-small"
chunk_size = 512
over_lap_size = 50 #put zero for non overlap
multi_process = True
normalize_embeddings = True
persist_directory = r"/content/drive/MyDrive/chunking/chunking_chroma_store"
collection_name = "ct_without_emb"

textsplitter = RecursiveCharacterTextSplitter
docs_processed = text_chunker.TextSplitters(RAW_KNOWLEDGE_BASE,textsplitter, chunk_size = chunk_size ,over_lap_size =over_lap_size)
print(len(docs_processed))

KNOWLEDGE_VECTOR_DATABASE = text_chunker.create_embedding_and_vectorstore(EMBEDDING_MODEL_NAME, multi_process, normalize_embeddings, docs_processed, persist_directory,collection_name )

#retrieved_docs =text_chunker.retrieve_content_documents(persist_directory, collection_name)
retrieved_content = text_chunker.retrieve_content_dic(persist_directory, collection_name)
retrieved_content

Processing PDF Pages:   0%|          | 0/32 [00:00<?, ?it/s]

32
2
{'source': '/content/drive/MyDrive/chunking/ds (1).pdf', 'page_number': 2, '/Author': '', '/Title': '', '/Subject': '', '/Creator': 'LaTeX with hyperref package', '/Producer': 'pdfTeX-1.40.19', '/Keywords': '', '/CreationDate': "D:20190814093040+02'00'", '/ModDate': "D:20190814093040+02'00'", '/Trapped': '/False', '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0', 'page': 1}
{'chunk_size': 512, 'chunk_overlap': 50, 'add_start_index': True, 'strip_whitespace': True, 'separators': ['']}
168


[{'id': '007bcd66-b3ee-4167-9de8-6642e1fc2aa5',
  'metadata': {'/Author': '',
   '/CreationDate': "D:20190814093040+02'00'",
   '/Creator': 'LaTeX with hyperref package',
   '/Keywords': '',
   '/ModDate': "D:20190814093040+02'00'",
   '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
   '/Producer': 'pdfTeX-1.40.19',
   '/Subject': '',
   '/Title': '',
   '/Trapped': '/False',
   'page': 30,
   'page_number': 31,
   'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
   'start_index': 924},
  'document': '20:89–97, 2004.\n[7] Antonin Chambolle, Vicent Caselles, Daniel Cremers, Matteo Novaga, and Thomas Pock. An intro-\nduction to total variation for image analysis. Theoretical foundations and numerical methods for sparse\nrecovery , 9(263-340):227, 2010.\n[8] Antonin Chambolle and Thomas Pock. An introduction to continuous optimization for imaging. Acta\nNumerica , 25:161–319, 2016.\n[9] S.S. Chen, D.L. Donoho, and M.A. Sa

In [ ]:
#Markdown TextSplitter



In [97]:
#Markdown TextSplitter without tokenizer

pdf_path = r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
pdf_content =   text_chunker.extract_pdf_content_with_metadata(pdf_path)
RAW_KNOWLEDGE_BASE =  text_chunker.create_langchain_document_objects(pdf_content)


EMBEDDING_MODEL_NAME = "thenlper/gte-small"
chunk_size = 512
over_lap_size = 50 #put zero for non overlap
multi_process = True
normalize_embeddings = True
persist_directory = r"/content/drive/MyDrive/chunking/chunking_chroma_store"
collection_name = "md_without_emb"

textsplitter = MarkdownTextSplitter

docs_processed = text_chunker.TextSplitters(RAW_KNOWLEDGE_BASE,textsplitter)
print(len(docs_processed))

KNOWLEDGE_VECTOR_DATABASE = text_chunker.create_embedding_and_vectorstore(EMBEDDING_MODEL_NAME, multi_process, normalize_embeddings, docs_processed, persist_directory,collection_name )

#retrieved_docs =text_chunker.retrieve_content_documents(persist_directory, collection_name)
retrieved_content = text_chunker.retrieve_content_dic(persist_directory, collection_name)
retrieved_content

Processing PDF Pages:   0%|          | 0/32 [00:00<?, ?it/s]

{'strip_whitespace': True, 'add_start_index': True}
32


[{'id': '002bd966-d06e-481a-abbc-cb57ca7cff82',
  'metadata': {'/Author': '',
   '/CreationDate': "D:20190814093040+02'00'",
   '/Creator': 'LaTeX with hyperref package',
   '/Keywords': '',
   '/ModDate': "D:20190814093040+02'00'",
   '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0',
   '/Producer': 'pdfTeX-1.40.19',
   '/Subject': '',
   '/Title': '',
   '/Trapped': '/False',
   'page': 6,
   'page_number': 7,
   'source': '/content/drive/MyDrive/chunking/ds (1).pdf',
   'start_index': 2730},
  'document': 'P∈U(a,b)⟨C,P⟩def.=∑\ni,jCi,jPi,j. (1.11)\nThis is a linear program (see Chapter ??), and as is usually the case with such programs, its solutions are\nnot necessarily unique.\n7'},
 {'id': '015460d5-3af2-4388-95f1-29d2b6fac52a',
  'metadata': {'/Author': '',
   '/CreationDate': "D:20190814093040+02'00'",
   '/Creator': 'LaTeX with hyperref package',
   '/Keywords': '',
   '/ModDate': "D:20190814093040+02'00'",
   '/PTEX.Fu

In [105]:
#SemanticChunker without tokenizer

pdf_path = r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
pdf_content =   text_chunker.extract_pdf_content_with_metadata(pdf_path)
RAW_KNOWLEDGE_BASE =  text_chunker.create_langchain_document_objects(pdf_content)


EMBEDDING_MODEL_NAME = "thenlper/gte-small"

multi_process = True
normalize_embeddings = True
persist_directory = r"/content/drive/MyDrive/chunking/chunking_chroma_store"
collection_name = "SM_without_emb"

embed_model = FastEmbedEmbeddings(model_name = text_embedding_model_name)
semantic_chunker = SemanticChunker(embed_model, breakpoint_threshold_type=breakpoint_threshold_type)

textsplitter =SemanticChunker

docs_processed = text_chunker.TextSplitters(RAW_KNOWLEDGE_BASE,textsplitter)
print(len(docs_processed))

KNOWLEDGE_VECTOR_DATABASE = text_chunker.create_embedding_and_vectorstore(EMBEDDING_MODEL_NAME, multi_process, normalize_embeddings, docs_processed, persist_directory,collection_name )

#retrieved_docs =text_chunker.retrieve_content_documents(persist_directory, collection_name)
retrieved_content = text_chunker.retrieve_content_dic(persist_directory, collection_name)
retrieved_content

Processing PDF Pages:   0%|          | 0/32 [00:00<?, ?it/s]

{}


TypeError: SemanticChunker.__init__() missing 1 required positional argument: 'embeddings'

In [102]:
from langchain.vectorstores import Chroma

def retrieve_content(persist_directory, collection_name):
    # Initialize the Chroma database for the specific collection
    knowledge_vector_database = Chroma(
        persist_directory=persist_directory,
        collection_name=collection_name,
    )

    # Retrieve documents from the collection
    knowledge_data = KNOWLEDGE_VECTOR_DATABASE.get()

    organized_data = []

    for i in range(len(knowledge_data['ids'])):
        data_dict = {
            'id': knowledge_data['ids'][i],
            'metadata': knowledge_data['metadatas'][i],
            'document': knowledge_data['documents'][i]
        }
        organized_data.append(data_dict)


    return organized_data

# Example usage

persist_directory = r"/content/drive/MyDrive/chunking/chunking_chroma_store"
collection_name = "rct"


# Retrieve content
retrieved_docs = retrieve_content(persist_directory, collection_name)

print(retrieved_docs)


[{'id': '0000a089-63ad-4055-849f-9efe4f606522', 'metadata': {'/Author': '', '/CreationDate': "D:20190814093040+02'00'", '/Creator': 'LaTeX with hyperref package', '/Keywords': '', '/ModDate': "D:20190814093040+02'00'", '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0', '/Producer': 'pdfTeX-1.40.19', '/Subject': '', '/Title': '', '/Trapped': '/False', 'page': 29, 'page_number': 30, 'source': '/content/drive/MyDrive/chunking/ds (1).pdf', 'start_index': 449}, 'document': 'P∈U(a,b)⟨P,C(ℓ)⟩−εH(P) where (1.86)\nC(ℓ)def.=∇ED,D′(P(ℓ)) =−D′TP(ℓ)D,\nwhich can be interpreted as a mirror-descent scheme [ ?]. Each update can thus be solved using Sinkhorn\niterations (1.51) with cost C(ℓ). Figure (1.17) illustrates the use of this entropic Gromov-Wasserstein to\ncompute soft maps between domains.\n30'}, {'id': '00d77c7b-7f0b-49c7-88cf-131d064f0c6a', 'metadata': {'/Author': '', '/CreationDate': "D:20190814093040+02'00'", '/Creator': 'LaTeX wit